# 1. Data loading

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras.utils import to_categorical
import numpy as np

filename = 'output.csv' # nome do dado de entrada
df = pd.read_csv(filename) # leitura do dado de entrada

In [ ]:
# indata 
training_data = df[['WELL','CALI','DEPTH_MD','GR','LITH','NPHI','RDEP','RHOB','RMED']].copy()
lith_num = {30000: 1,65030: 2,65000: 3,80000: 4,74000: 5,70000: 6,70032: 7,88000: 8,86000: 9,99000: 10,90000: 11,93000: 12}
training_data['LITH_SI'] = training_data['LITH'].map(lith_num)
blind = training_data[training_data['WELL'] == '16/2-16'] #seleciona um poço apenas do dado
training_data = training_data[training_data['WELL'] != '16/2-16'] #remove o poço do dado
training_data['WELL'].unique()
features = ['CALI','GR','NPHI','RDEP','RHOB','RMED']
#Select Facies
y = training_data['LITH_SI']
# well curves
X = training_data[features]
#split data to train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# Blind test well
y_blind = blind['LITH_SI']
X_blind = blind[features]
X_blind_stnd = sc.transform(X_blind)
training_features = ['Ss','Ss/Sh','Sh','M','D','L','Ch','A','H','T','C']
# list_blind_full =   ['Ss','Ss/Sh','Sh','M','L','Ch','A','T','C']
list_blind_full =   ['Ss','Ss/Sh','Sh','M','NaN','L','NaN','A','T']
sample_size = X_train.shape[0] # number of samples in train set
time_steps  = X_train.shape[1] # number of features in train set
input_dimension = 1               # each feature is represented by 1 number

train_data_reshaped = X_train.reshape(sample_size,time_steps,input_dimension)

# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train_encoded = to_categorical(y_train - 1, num_classes=num_classes)
y_test_encoded = to_categorical(y_test - 1, num_classes=num_classes)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dropout, Dense, LeakyReLU, Reshape, Conv1DTranspose, Concatenate, Embedding

def build_discriminator(input_shape=(6, 1), num_classes=11):
    data_input = Input(shape=input_shape)
    label_input = Input(shape=(num_classes,))
    labels_reshaped = Reshape((1, num_classes))(label_input)
    labels_reshaped = tf.keras.layers.UpSampling1D(size=6)(labels_reshaped)
    concatenated = Concatenate(axis=-1)([data_input, labels_reshaped])
    
    x = Conv1D(64, 2, strides=2, padding='same', activation='relu')(concatenated)
    x = Dropout(0.4)(x)  # Dropout added
    x = Flatten()(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model([data_input, label_input], output, name="discriminator")
    # Compile the discriminator model
    optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    
    return model

def build_generator(latent_dim, num_classes):
    # Inputs
    noise_input = Input(shape=(latent_dim,))
    label_input = Input(shape=(num_classes,))
    
    # Merging noise and label inputs
    merged_input = Concatenate()([noise_input, label_input])
    
    # Intermediate layer
    x = Dense(60)(merged_input)  # Adjusted number of units
    x = LeakyReLU(alpha=0.2)(x)
    
    # Branch for well profiles (Assuming well profiles are time-series data)
    x_profiles = Reshape((6, 10))(x)
    x_profiles = Conv1DTranspose(64, 3, strides=1, padding='same')(x_profiles)
    x_profiles = LeakyReLU(alpha=0.2)(x_profiles)
    x_profiles = Conv1DTranspose(32, 3, strides=1, padding='same')(x_profiles)
    x_profiles = LeakyReLU(alpha=0.2)(x_profiles)
    well_profiles_output = Conv1D(1, 3, activation='linear', padding='same')(x_profiles)
    well_profiles_output = Reshape((6, 1))(well_profiles_output)
    
    # Branch for lithology predictions
    x_lithology = Dense(num_classes, activation='softmax')(x)  # Softmax for multi-class classification
    
    return Model([noise_input, label_input], [well_profiles_output, x_lithology], name="generator")
    # return Model([noise_input, label_input], output, name="generator")

def build_gan(generator, discriminator):
    # Make the discriminator not trainable when compiling GAN
    discriminator.trainable = False
    noise = Input(shape=(latent_dim,))
    labels = Input(shape=(num_classes,))
    gen_data, gen_labels = generator([noise, labels])
    valid = discriminator([gen_data, labels])
    gan = Model([noise, labels], [valid, gen_labels])
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    gan.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=opt)
    return gan

import numpy as np

def get_real_samples(dataset, labels, n_samples):
    # Select random instances
    idx = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    y = np.ones((n_samples, 1))  # Real samples label: 1
    labels = labels[idx]
    return X, labels, y

# Function to generate fake samples
def generate_fake_samples(generator, latent_dim, num_classes, n_samples):
    # Generate points in latent space
    noise = np.random.randn(latent_dim * n_samples)
    noise = noise.reshape(n_samples, latent_dim)
    # Generate random labels
    sampled_labels = np.random.randint(0, num_classes, n_samples)
    sampled_labels = to_categorical(sampled_labels, num_classes=num_classes)
    # Generate fake samples
    X, y_labels = generator.predict([noise, sampled_labels])
    y = np.zeros((n_samples, 1))  # Fake samples label: 0
    return X, y_labels, y

# Training the GAN
def train_gan(generator, discriminator, gan, dataset, labels, latent_dim, n_epochs=1000, n_batch=128):
    half_batch = n_batch // 2
    for i in range(n_epochs):
        # Get randomly selected 'real' samples and their labels
        X_real, labels_real, y_real = get_real_samples(dataset, labels, half_batch)
        y_real_labels = labels_real  # Additional real labels for lithology
        # Generate 'fake' examples and random labels
        X_fake, labels_fake, y_fake = generate_fake_samples(generator, latent_dim, num_classes, half_batch)
        y_fake_labels = labels_fake  # Fake labels for lithology

        # Train the discriminator
        discriminator.train_on_batch([X_real, labels_real], [y_real, y_real_labels])
        discriminator.train_on_batch([X_fake, labels_fake], [y_fake, y_fake_labels])
        # Prepare points in latent space as input for the generator
        noise = np.random.randn(latent_dim * n_batch)
        sampled_labels = np.random.randint(0, num_classes, n_batch)
        sampled_labels = to_categorical(sampled_labels, num_classes=num_classes)
        noise = noise.reshape(n_batch, latent_dim)
        # The generator wants the discriminator to label the generated samples as valid (1)
        y_gan = np.ones((n_batch, 1))
        y_gan_labels = sampled_labels  # Generator aims to fool the discriminator with these labels
        # Update the generator via the discriminator's error
        gan.train_on_batch([noise, sampled_labels], [y_gan, y_gan_labels])
        if (i+1) % (n_epochs // 10) == 0:
            summarize_performance(i, generator, discriminator, dataset, labels, latent_dim)

# Summarize the performance of the discriminator and generator
def summarize_performance(epoch, generator, discriminator, dataset, labels, latent_dim, n_samples=100):
    X_real, labels_real, y_real = get_real_samples(dataset, labels, n_samples)
    acc_real = discriminator.evaluate([X_real, labels_real], y_real, verbose=0)
    X_fake, labels_fake, y_fake = generate_fake_samples(generator, latent_dim, num_classes, n_samples)
    acc_fake = discriminator.evaluate([X_fake, labels_fake], y_fake, verbose=0)
    print(f"Epoch {epoch+1}, Accuracy Real: {acc_real*100}, Accuracy Fake: {acc_fake*100}")

# Assuming train_data_reshaped is your input data
n_timesteps = train_data_reshaped.shape[1] #
n_features  = train_data_reshaped.shape[2] # 
data_shape = (n_timesteps, n_features)
# Assuming 'dataset' is your loaded well log data and it's preprocessed (e.g., scaled)
latent_dim = 100 # This is an example; adjust based on your specific needs
# Build the models
generator = build_generator(latent_dim, num_classes)
#TODO: colocar as dimensoes como entrada, remover hardcoded
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)
generator.summary()
discriminator.summary()
gan.summary()

# Train GAN
train_gan(generator, discriminator, gan, train_data_reshaped, y_train_encoded, latent_dim)

In [ ]:
def generate_synthetic_data(generator, latent_dim, n_samples, n_classes):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    sampled_labels = np.random.randint(0, num_classes, n_samples)
    sampled_labels = to_categorical(sampled_labels, num_classes=num_classes)
    X, y_labels = generator.predict([x_input, sampled_labels])
    y_labels = np.argmax(y_labels, axis=-1)
    return X, y_labels

n_synthetic_samples = 5000  # Number of synthetic samples to generate
synthetic_data, synthetic_labels = generate_synthetic_data(generator, latent_dim, n_synthetic_samples, n_classes=len(np.unique(y_train)))
#TODO: Undo the reshape of synthetic data (5000, 6, 1) -> (5000, 6)
synthetic_data = np.squeeze(synthetic_data, axis=-1)

# Combine real and synthetic data
X_augmented = np.vstack((X_train, synthetic_data))
y_augmented = np.hstack((y_train, synthetic_labels))

# Step 4: Train a Classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_augmented, y_augmented)

# Step 5: Evaluate the Classifier
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
def build_conv1D_model():

    n_timesteps = train_data_reshaped.shape[1] #
    n_features  = train_data_reshaped.shape[2] # 
       
    
    model = keras.Sequential(name="model_conv1D")
    
    # 1st layer
    ks = 2
    model.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_1"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_2"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_3"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    model.add(keras.layers.Conv1D(filters=200, kernel_size=ks, strides=1, padding='valid', activation='relu', name="Conv1D_4"))
    model.add(keras.layers.MaxPooling1D(pool_size=1))
    
    #model.add(keras.layers.MaxPooling1D(pool_size=1, name="MaxPooling1D_fisrt"))
    
    # Dense
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(12, activation='softmax'))


    optimizer_aux = tf.keras.optimizers.Adam()
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer_aux ,metrics = ['accuracy'])
    
    return model

model_conv1D = build_conv1D_model()
model_conv1D.summary()


In [ ]:
earlystoping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                patience=5,
                                                verbose=1,
                                                mode='auto',
                                                restore_best_weights=True)
checkpoint_filepath = 'weights.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                      save_weights_only=True,
                                                      monitor='val_accuracy',
                                                      mode='max',
                                                      verbose=1,
                                                      save_best_only=True)

In [ ]:
history_cnn = model_conv1D.fit(train_data_reshaped, y_train, validation_data = (test_data_reshaped,y_test),
                           batch_size = 512, 
                           callbacks = [model_checkpoint,earlystoping],
                           epochs = 1000,
                           verbose=1)

In [ ]:
plt.plot(history_cnn.history['loss'])
plt.plot(history_cnn.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])


In [ ]:
plt.plot(history_cnn.history['accuracy'])
plt.plot(history_cnn.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
pred_test_cnn = model_conv1D.predict(test_data_reshaped)
pred_test_cnn = tf.argmax(pred_test_cnn, axis=1)
test_loss, test_acc = model_conv1D.evaluate(test_data_reshaped,  y_test, verbose=2)
print(classification_report(y_test, pred_test_cnn, target_names=training_features))
cm_test_cnn = confusion_matrix(y_test, pred_test_cnn)
plot_confusion_matrix(cm_test_cnn, training_features, normalize=True)

In [ ]:
microF1_test_cnn = f1_score(y_test, pred_test_cnn, average='micro')
print('Test Macro f1 score:', microF1_test_cnn)
#
X_blind_reshaped = X_blind_stnd.reshape(X_blind_stnd.shape[0],X_blind_stnd.shape[1],1)
aux = model_conv1D.predict(X_blind_reshaped)
pred_blind_cnn = tf.argmax(aux, axis=1)

In [ ]:
# Blind CM
print(classification_report(y_blind, pred_blind_cnn))
cm_cnn = confusion_matrix(y_blind, pred_blind_cnn)
list_blind_full =   ['Ss','Ss/Sh','Sh','M','NaN','L','NaN','A','T']
plot_confusion_matrix(cm_cnn, list_blind_full, normalize=True)

In [ ]:
microF1_blind_cnn = f1_score(y_blind, pred_blind_cnn, average='micro')
print('Test Macro f1 score:', microF1_blind_cnn)

In [ ]:
from rbflayer import RBFLayer, InitCentersRandom
def build_conv1D_rbf_model():
    #
    n_timesteps = train_data_reshaped.shape[1] #
    n_features  = train_data_reshaped.shape[2] # 
    #
    model_rbf = keras.Sequential(name="model_conv1D_rbf")
    # 1st layer
    ks = 1
    mp=1
    num_filters=128
    model_rbf.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model_rbf.add(keras.layers.Conv1D(filters=num_filters, kernel_size=ks, activation='relu', name="Conv1D_1"))
    model_rbf.add(keras.layers.Conv1D(filters=num_filters, kernel_size=ks, activation='relu', name="Conv1D_2"))
    model_rbf.add(keras.layers.MaxPooling1D(pool_size=mp))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.BatchNormalization())
    # # 2nd layer
    model_rbf.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model_rbf.add(keras.layers.Conv1D(filters=num_filters, kernel_size=ks, activation='relu', name="Conv1D_3"))
    model_rbf.add(keras.layers.Conv1D(filters=num_filters, kernel_size=ks, activation='relu', name="Conv1D_4"))
    model_rbf.add(keras.layers.MaxPooling1D(pool_size=mp))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.BatchNormalization())
    # # 3rd layer
    model_rbf.add(keras.layers.Input(shape=(n_timesteps,n_features)))
    model_rbf.add(keras.layers.Conv1D(filters=num_filters, kernel_size=ks, activation='relu', name="Conv1D_5"))
    model_rbf.add(keras.layers.Conv1D(filters=num_filters, kernel_size=ks, activation='relu', name="Conv1D_6"))
    model_rbf.add(keras.layers.MaxPooling1D(pool_size=mp))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.BatchNormalization())
    model_rbf.add(RBFLayer(num_filters,
                           #TODO: Implement the function below
                        #    initializer=InitCentersRandom(X_blind_reshaped),
                           betas=2.0,
                           input_shape=(1,)))
    model_rbf.add(keras.layers.Flatten())
    model_rbf.add(keras.layers.Dense(512, activation='relu'))
    model_rbf.add(keras.layers.Dropout(0.2))
    model_rbf.add(keras.layers.Dense(12, activation='softmax'))


    optimizer_aux = tf.keras.optimizers.Adam()
    model_rbf.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer_aux ,metrics = ['accuracy'])
    
    return model_rbf

model_conv1D_rbf = build_conv1D_rbf_model()
model_conv1D_rbf.summary()


In [ ]:
history_rbf = model_conv1D_rbf.fit(train_data_reshaped, y_train, validation_data = (test_data_reshaped,y_test),
                           batch_size = 512, 
                           callbacks = [model_checkpoint,earlystoping],
                           epochs = 1000,
                           verbose=1)

In [ ]:
plt.plot(history_rbf.history['loss'])
plt.plot(history_rbf.history['val_loss'])
plt.title('model loss CNN (RBF)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

In [ ]:
plt.plot(history_rbf.history['accuracy'])
plt.plot(history_rbf.history['val_accuracy'])
plt.title('model accuracy CNN (RBF)')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
pred_test_rbf = model_conv1D_rbf.predict(test_data_reshaped)
pred_test_rbf = tf.argmax(pred_test_rbf, axis=1)
print(classification_report(y_test, pred_test_rbf))
cm_test_rbf = confusion_matrix(y_test, pred_test_rbf)
plot_confusion_matrix(cm_test_rbf, training_features, normalize=True)

In [ ]:
microF1_test_rbf = f1_score(y_test, pred_test_rbf, average='micro')
print('Test Macro f1 score:', microF1_test_rbf)
pred_blind_rbf = model_conv1D_rbf.predict(X_blind_reshaped)
pred_blind_rbf = tf.argmax(pred_blind_rbf, axis=1)

In [ ]:
print(classification_report(y_blind, pred_blind_rbf))
cm_rbf = confusion_matrix(y_blind, pred_blind_rbf)
list_blind_full =   ['Ss','Ss/Sh','Sh','M','L','NaN','A','T']
plot_confusion_matrix(cm_rbf, list_blind_full, normalize=True)

In [ ]:
from tensorflow.keras.layers import LSTM
#
def build_lstm_model():
    n_timesteps = train_data_reshaped.shape[1]
    n_features = train_data_reshaped.shape[2]

    model_lstm = keras.Sequential(name="model_lstm")
    
    # 1st layer
    num_lstm_units = 128
    model_lstm.add(keras.layers.Input(shape=(n_timesteps, n_features)))
    model_lstm.add(LSTM(units=num_lstm_units, return_sequences=True, activation='relu', name="LSTM_1"))
    model_lstm.add(LSTM(units=num_lstm_units, return_sequences=True, activation='relu', name="LSTM_2"))
    model_lstm.add(keras.layers.Dropout(0.2))
    model_lstm.add(keras.layers.BatchNormalization())
    
    # 2nd layer
    model_lstm.add(LSTM(units=num_lstm_units, return_sequences=True, activation='relu', name="LSTM_3"))
    model_lstm.add(LSTM(units=num_lstm_units, return_sequences=True, activation='relu', name="LSTM_4"))
    model_lstm.add(keras.layers.Dropout(0.2))
    model_lstm.add(keras.layers.BatchNormalization())
    
    # 3rd layer
    model_lstm.add(LSTM(units=num_lstm_units, return_sequences=True, activation='relu', name="LSTM_5"))
    model_lstm.add(LSTM(units=num_lstm_units, return_sequences=True, activation='relu', name="LSTM_6"))
    model_lstm.add(keras.layers.Dropout(0.2))
    model_lstm.add(keras.layers.BatchNormalization())
    
    # Dense
    model_lstm.add(keras.layers.Flatten())
    model_lstm.add(keras.layers.Dense(512, activation='relu'))
    model_lstm.add(keras.layers.Dropout(0.2))
    model_lstm.add(keras.layers.Dense(12, activation='softmax'))

    optimizer_aux = tf.keras.optimizers.Adam()
    model_lstm.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_aux, metrics=['accuracy'])

    return model_lstm

model_lstm = build_lstm_model()
model_lstm.summary()


In [ ]:
history_lstm = model_lstm.fit(train_data_reshaped, y_train, validation_data = (test_data_reshaped,y_test),
                           batch_size = 512, 
                           callbacks = [model_checkpoint,earlystoping],
                           epochs = 1000,
                           verbose=1)

In [ ]:
plt.plot(history_lstm.history['loss'])
plt.plot(history_lstm.history['val_loss'])
plt.title('model loss LSTM')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

In [ ]:
plt.plot(history_lstm.history['accuracy'])
plt.plot(history_lstm.history['val_accuracy'])
plt.title('model accuracy LSTM')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
pred_test_lstm = model_lstm.predict(test_data_reshaped)
pred_test_lstm = tf.argmax(pred_test_lstm, axis=1)
#
print(classification_report(y_test, pred_test_lstm))
cm_test_lstm = confusion_matrix(y_test, pred_test_lstm)
plot_confusion_matrix(cm_test_lstm, training_features, normalize=True)

In [ ]:
microF1_test_lstm = f1_score(y_test, pred_test_lstm, average='micro')
print('Test Macro f1 score:', microF1_test_lstm)

In [ ]:
pred_blind_lstm = model_lstm.predict(X_blind_reshaped)
pred_blind_lstm = tf.argmax(pred_blind_lstm, axis=1)
print(classification_report(y_blind, pred_blind_lstm))
cm_lstm = confusion_matrix(y_blind, pred_blind_lstm)
list_blind_full =   ['Ss','Ss/Sh','Sh','M','NaN','L','NaN','A','T','NaN']
plot_confusion_matrix(cm_lstm, list_blind_full, normalize=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GRU, MaxPooling1D, Dropout, BatchNormalization, Flatten, Dense

def build_gru_model():
    n_timesteps = train_data_reshaped.shape[1]
    n_features = train_data_reshaped.shape[2]
    
    model_gru = Sequential(name="model_gru")
    
    # GRU Layers
    num_units = 128
    model_gru.add(Input(shape=(n_timesteps, n_features)))
    model_gru.add(GRU(units=num_units, return_sequences=True, activation='relu', name="GRU_1"))
    model_gru.add(GRU(units=num_units, return_sequences=True, activation='relu', name="GRU_2"))
    model_gru.add(MaxPooling1D(pool_size=1))
    model_gru.add(Dropout(0.2))
    model_gru.add(BatchNormalization())
    
    model_gru.add(GRU(units=num_units, return_sequences=True, activation='relu', name="GRU_3"))
    model_gru.add(GRU(units=num_units, return_sequences=True, activation='relu', name="GRU_4"))
    model_gru.add(MaxPooling1D(pool_size=1))
    model_gru.add(Dropout(0.2))
    model_gru.add(BatchNormalization())
    
    model_gru.add(GRU(units=num_units, return_sequences=True, activation='relu', name="GRU_5"))
    model_gru.add(GRU(units=num_units, return_sequences=True, activation='relu', name="GRU_6"))
    model_gru.add(MaxPooling1D(pool_size=1))
    model_gru.add(Dropout(0.2))
    model_gru.add(BatchNormalization())
    
    # Flatten Layer
    model_gru.add(Flatten())
    
    # Dense Layers
    model_gru.add(Dense(512, activation='relu'))
    model_gru.add(Dropout(0.2))
    model_gru.add(Dense(12, activation='softmax'))

    # Model Compilation
    optimizer_aux = tf.keras.optimizers.Adam()
    model_gru.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_aux, metrics=['accuracy'])
    
    return model_gru

model_gru = build_gru_model()
model_gru.summary()

In [ ]:
history_gru = model_gru.fit(train_data_reshaped, y_train, validation_data = (test_data_reshaped,y_test),
                           batch_size = 512, 
                           callbacks = [model_checkpoint,earlystoping],
                           epochs = 1000,
                           verbose=1)

In [ ]:
plt.plot(history_gru.history['loss'])
plt.plot(history_gru.history['val_loss'])
plt.title('model loss GRU')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])

In [ ]:

plt.plot(history_gru.history['accuracy'])
plt.plot(history_gru.history['val_accuracy'])
plt.title('model accuracy GRU')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
pred_test_gru = model_gru.predict(test_data_reshaped)
pred_test_gru = tf.argmax(pred_test_gru, axis=1)
print(classification_report(y_test, pred_test_gru))
cm_test_gru = confusion_matrix(y_test, pred_test_gru)
plot_confusion_matrix(cm_test_gru, training_features, normalize=True)

In [ ]:
microF1_test_gru = f1_score(y_test, pred_test_gru, average='micro')
print('Test Macro f1 score:', microF1_test_gru)

In [ ]:
pred_blind_gru = model_gru.predict(X_blind_reshaped)
pred_blind_gru = tf.argmax(pred_blind_gru, axis=1)
print(classification_report(y_blind, pred_blind_gru))
cm_gru = confusion_matrix(y_blind, pred_blind_gru)
list_blind_full =   ['Ss','Ss/Sh','Sh','M','NaN','L','NaN','A','T','NaN']
plot_confusion_matrix(cm_gru, list_blind_full, normalize=True)

# 10. Model performance evaluation

I will use the diagnosis of confusion matrix from train data set to evaluate the model performance. The diagnosis of confusion matrix points how much percentage of the stone is correctly predicted.

In [ ]:
### To create a data frame recording the correct prediction (normalized) of 
### facies for each machine learning algorithm

mod_test_list = ['CNN','CNN-RBF','LSTM','GRU']
cm_test_list = [cm_test_cnn, cm_test_rbf, cm_test_lstm, cm_test_gru]
face_test_list = training_features
pred_test_df = pd.DataFrame(index=training_features, columns=mod_test_list)

for mod in mod_test_list:
    
    col_index = int(mod_test_list.index(mod))
    cm = cm_test_list[col_index]
    
    for face in face_test_list:
        row_index = training_features.index(face)
        #print(face, row_index, col_index)
        pred_test_df.iloc[row_index, col_index] = cm[row_index][row_index]/sum(cm[row_index])
        

### add the accuracy factor
df_1 = pd.DataFrame([[microF1_test_cnn, 
                      microF1_test_rbf,
                      microF1_test_lstm,
                      microF1_test_gru]], index=['Accuracy'], columns=mod_test_list)    


pred_test_conc = pd.concat([pred_test_df,df_1])
pred_test_conc

In [ ]:
X_ind = np.arange(pred_test_df.shape[0])
(pred_df_index_list) = training_features
aux=0.1
plt.figure(figsize=(10,5))
plt.bar(X_ind, pred_test_df['CNN'], color='blue', width=aux)
plt.bar(X_ind+0.1, pred_test_df['CNN-RBF'], color='red', width=aux)
plt.bar(X_ind+0.2, pred_test_df['LSTM'], color='green', width=aux)
plt.bar(X_ind+0.3, pred_test_df['GRU'], color='black', width=aux)

plt.xticks(X_ind, pred_df_index_list)
plt.xlabel('Facies')
plt.ylabel('Correct predictions')
plt.legend(labels=mod_test_list)
plt.savefig('canada_performance_evaluation_test_data.pdf',bbox_inches='tight')
plt.show()

# 11. Calssifier evluation using blind test well

I will use the same method shown in item4 for evaluation.

In [ ]:
### To create a data frame recording the correct prediction (normalized) of facies of blind test well for each machine learning algorithm

# blind_class  = ['Sandstone',
#                   'Sandstone/Shale',
#                   'Shale',
#                   'Marl',
#                   'Limestone',
#                   'Chalk',
#                   'Anhydrite',
#                   'Tuff']
list_blind_full =   ['Ss','Ss/Sh','Sh','M','L','A','NaN','T']
mod_list = ['CNN','CNN-RBF','LSTM','GRU']
cm_list = [cm_cnn, cm_rbf, cm_lstm, cm_gru]
pred_df = pd.DataFrame(index=list_blind_full, columns=mod_list)

for mod in mod_list:
    col_index = int(mod_list.index(mod))
    cm = cm_list[col_index]
    
    for face in list_blind_full:
        
        row_index = list_blind_full.index(face)
        #print(face, row_index, col_index)
        pred_df.iloc[row_index, col_index] = cm[row_index][row_index]/sum(cm[row_index])



In [ ]:
X_ind = np.arange(pred_df.shape[0])

aux=0.1
plt.figure(figsize=(10,5))
plt.bar(X_ind, pred_df['CNN'], color='blue', width=aux)
plt.bar(X_ind+0.1, pred_df['CNN-RBF'], color='red', width=aux)
plt.bar(X_ind+0.3, pred_df['LSTM'], color='green', width=aux)
plt.bar(X_ind+0.4, pred_df['GRU'], color='black', width=aux)
plt.xticks(X_ind, list_blind_full)
plt.xlabel('Facies')
plt.ylabel('Correct predictions')
plt.legend(labels=mod_list)
plt.savefig('canada_performance_evaluation_blind_data.pdf',bbox_inches='tight')
plt.show()

# 12. Plot the predicted facies for comparison**

In [ ]:
blind = blind.copy()
blind['CNN'] = pred_blind_cnn
blind['CNN-RBF'] = pred_blind_rbf
blind['LSTM'] = pred_blind_lstm
blind['GRU'] = pred_blind_gru


blind.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

facies_colors = ['bisque',
                 'darkorange',
                 'darkgoldenrod',
                 'peachpuff',
                 'beige',
                 'white',
                 'red']

blind_class  = ['Ss',
                  'Ss/Sh',
                  'Sh',
                  'M',
                  'L',
                  'A',
                  'T']

def compare_facies_plot(logs, facie1, facie2, facie3, facie4, facies_colors):
    #make sure logs are sorted by depth
    logs = logs.sort_values(by='DEPTH_MD')
    cmap_facies = colors.ListedColormap(facies_colors[0:len(facies_colors)], 'indexed')
    num_colors = 7
    ztop=logs.DEPTH_MD.min(); zbot=logs.DEPTH_MD.max()
    #
    cluster0 = np.repeat(np.expand_dims(logs['LITH_SI'].values,1), 100, 1)
    cluster1 = np.repeat(np.expand_dims(logs[facie1].values,1), 100, 1)
    cluster2 = np.repeat(np.expand_dims(logs[facie2].values,1), 100, 1)
    cluster3 = np.repeat(np.expand_dims(logs[facie3].values,1), 100, 1)
    cluster4 = np.repeat(np.expand_dims(logs[facie4].values,1), 100, 1)
    # cluster5 = np.repeat(np.expand_dims(logs[compare5].values,1), 100, 1)
    # cluster6 = np.repeat(np.expand_dims(logs[compare6].values,1), 100, 1)
    # cluster7 = np.repeat(np.expand_dims(logs[compare7].values,1), 100, 1)
    #
    f, ax = plt.subplots(nrows=1, ncols=11, figsize=(18, 15))
    ax[0].plot(logs.CALI, logs.DEPTH_MD, '-',color='red')
    ax[1].plot(logs.GR, logs.DEPTH_MD, '-',color='blue')
    ax[2].plot(logs.NPHI, logs.DEPTH_MD, '-', color='red')
    ax[3].plot(np.log10(logs.RDEP), logs.DEPTH_MD, '-', color='green')
    ax[4].plot(logs.RHOB, logs.DEPTH_MD, '--', color='blue')
    ax[5].plot(np.log10(logs.RMED), logs.DEPTH_MD, '-', color='black')
    # ax[6].plot(logs.PEF, logs.DEPTH_MD, '-', color='black')
    im0 = ax[6].imshow(cluster0, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    im1 = ax[7].imshow(cluster1, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    im2 = ax[8].imshow(cluster2, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    im3 = ax[9].imshow(cluster3, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    im4 = ax[10].imshow(cluster4, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im4 = ax[12].imshow(cluster5, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im4 = ax[13].imshow(cluster6, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    # im4 = ax[14].imshow(cluster7, interpolation='none', aspect='auto',
    #                 cmap=cmap_facies,vmin=1,vmax=num_colors)
    
            
    divider = make_axes_locatable(ax[10])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im4, cax=cax)
    cbar.set_label((30*' ').join(blind_class))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
    
    for i in range(len(ax)-5): # 5 represents facies number
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=5)
    
    ax[0].set_xlabel("CALI")
    ax[0].set_xlim(8,16)
    
    ax[1].set_xlabel("GR")
    ax[1].set_xlim(0,200)
    
    ax[2].set_xlabel("NPHI")
    ax[2].set_xlim(0.45,-0.15)
    
    ax[3].set_xlabel("RDEP")
    # ax[3].set_xlim(0.2,200)
    
    ax[4].set_xlabel("RHOB")
    ax[4].set_xlim(1.95,2.95)
    
    ax[5].set_xlabel("RMED")
    # ax[5].set_xlim(0.2,200)
    
    # ax[6].set_xlabel("PEF")
    # ax[6].set_xlim(logs.PEF.min(),logs.PEF.max())
    
    ax[6].set_xlabel('Facies')
    ax[7].set_xlabel(facie1)
    ax[8].set_xlabel(facie2)
    ax[9].set_xlabel(facie3)
    ax[10].set_xlabel(facie4)
    # ax[12].set_xlabel(compare5)
    # ax[13].set_xlabel(compare6)
    # ax[14].set_xlabel(compare7)
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]); ax[5].set_yticklabels([]); ax[6].set_yticklabels([])
    ax[7].set_yticklabels([]); ax[8].set_yticklabels([]); ax[9].set_yticklabels([])
    ax[10].set_yticklabels([]); #ax[11].set_yticklabels([])#; ax[12].set_yticklabels([])
    # ax[13].set_yticklabels([]); ax[14].set_yticklabels([])
    
    
    ax[5].set_xticklabels([])
    ax[6].set_xticklabels([])
    ax[7].set_xticklabels([])
    ax[8].set_xticklabels([])
    ax[9].set_xticklabels([])
    ax[10].set_xticklabels([])
    #ax[11].set_xticklabels([])
    # ax[12].set_xticklabels([])
    # ax[13].set_xticklabels([])
    # ax[14].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['WELL'], fontsize=14,y=0.94)

In [ ]:
compare_facies_plot(blind, 'CNN','CNN-RBF', 'LSTM', 'GRU', facies_colors)